## 使用 `czsc` 进行选股
---


In [1]:
import sys
sys.path.insert(0, ".")
sys.path.insert(0, "..")

import czsc
print(czsc.__version__)

from datetime import datetime
from typing import List
import traceback
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from czsc.analyze import KlineAnalyze
# 导入聚宽数据

pd.set_option('display.max_rows', None)



0.5.9


In [ ]:
import tushare as ts
# 如果是第一次使用需要设置 token
ts.set_token("565ee6d69fb85cb0bc7fdfc8dea4b8ce1f18366f30e8d23a253bb9cf")
from czsc.data.ts import *


In [2]:
from czsc.data.bs import *

login success!
login respond error_code:0
login respond  error_msg:success


### 笔中枢三买选股
---

条件描述： 最近五笔走势，前三笔构成中枢，第四笔离开中枢，第五笔向下不回中枢



In [3]:
close = get_kline("000768.SZ", end_date="2020-12-31", freq="d", count=1)
print(close["close"][0])

36.68


/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
def selector_bi01(kline, res):
    """笔中枢三买选股"""
    
    try:
        ka = KlineAnalyze(kline, bi_mode="old", max_count=1000)
        points = ka.bi_list[-7:]

        if len(points) == 7 and points[-1]['fx_mark'] == "d":
            zs_g = min([x['bi'] for x in points[1:5] if x['fx_mark'] == 'g'])
            zs_d = max([x['bi'] for x in points[1:5] if x['fx_mark'] == 'd'])
            #close = get_kline(symbol, end_date=ka.fx_list[-1]["dt", freq=freq, count=1)
            if ka.latest_price > zs_g > zs_d and points[0]['bi'] < zs_d and ka.fx_list[-1]['fx_mark'] == 'g' and ka.fx_list[-1]["fx_low"] > ka.latest_price:
                res['symbol'] = ka.symbol
                res['is_match'] = True
                res['dt'] = ka.fx_list[-1]['dt']
                res['price'] = ka.latest_price
                res['fx'] = ka.fx_list[-1]
                res['bi'] = ka.bi_list[-1]
                res['fx_h'] = ka.fx_list[-1]["fx_high"]
                res['fx_l'] = ka.fx_list[-1]["fx_low"]
                res['笔中枢三买选股'] = True

    except:
        print("{} 分析失败".format(symbol))
        traceback.print_exc()
    return res

### 三笔回调构成三买
---

In [5]:
def selector_bi02(kline, res):
    """三笔回调构成三买"""

    try:
        ka = KlineAnalyze(kline, bi_mode="old", max_count=1000)
        if len(ka.bi_list) > 16 and ka.bi_list[-1]['fx_mark'] == 'd':
            points = ka.bi_list[-16:]
            fd1_points = [x['bi'] for x in points[0: 4]]
            fd2_points = [x['bi'] for x in points[3: 7]]
            fd3_points = [x['bi'] for x in points[6: 10]]
            fd4_points = [x['bi'] for x in points[9: 13]]
            fd5_points = [x['bi'] for x in points[12: 16]]
            fd1 = {"high": max(fd1_points), "low": min(fd1_points)}
            fd2 = {"high": max(fd2_points), "low": min(fd2_points)}
            fd3 = {"high": max(fd3_points), "low": min(fd3_points)}
            fd4 = {"high": max(fd4_points), "low": min(fd4_points)}
            fd5 = {"high": max(fd5_points), "low": min(fd5_points)}

            zs_g = min(fd1['high'], fd2['high'], fd3['high'])
            zs_d = max(fd1['low'], fd2['low'], fd3['low'])
            if zs_d < zs_g < fd5['low'] < fd4['high'] and fd4['high'] > max(fd1['high'], fd3['high']):
                res['symbol'] = ka.symbol
                res['is_match'] = True
                res['dt'] = ka.fx_list[-1]['dt']
                res['price'] = ka.latest_price
                res['fx'] = ka.fx_list[-1]
                res['bi'] = ka.bi_list[-1]
                res['fx_h'] = ka.fx_list[-1]["fx_high"]
                res['fx_l'] = ka.fx_list[-1]["fx_low"]
                res['三笔回调构成三买'] = True
    except:
        print(ka.fx_list)
        traceback.print_exc()
    return res

## 执行选股
---

In [6]:

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

# 使用聚宽数据在创业板综指上选股
# symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("zz500")
freq = "d"

results = []
for symbol in tqdm(symbols, desc="selector"):
    res = {"is_match": False, "fx": None, "bi": None, "dt": None, "price": None, "fx_h": None, "fx_l": None,
           "name": "selector_bi02", "symbol": None, "三笔回调构成三买": False, "笔中枢三买选股": False, "name": None}
    try:
        kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
        for selector in [selector_bi01, selector_bi02]:
            try:
                res = selector(kline, res)
            except:
                continue
        if res['is_match']:
            res['name'] = get_stock_info(symbol).code_name[0]
            results.append(res)
    except:
        traceback.print_exc()
        print("fail on: {}".format(symbol))

print("选股结果：")
zz500df = pd.DataFrame(results)
# df.to_excel("选股结果.xlsx", index=False)

<ipython-input-6-fe7c2d02d8a8>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for symbol in tqdm(symbols, desc="selector"):


selector:   0%|          | 0/500 [00:00<?, ?it/s]

选股结果：


In [ ]:

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

# 使用聚宽数据在创业板综指上选股
# symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("all")
freq = "d"

results = []
for symbol in tqdm(symbols, desc="selector"):
    res = {"is_match": False, "fx": None, "bi": None, "dt": None, "price": None, "fx_h": None, "fx_l": None,
           "name": "selector_bi02", "symbol": None, "三笔回调构成三买": False, "笔中枢三买选股": False, "name": None}
    try:
        kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
        for selector in [selector_bi01, selector_bi02]:
            try:
                res = selector(kline, res)
            except:
                continue
        if res['is_match']:
            res['name'] = get_stock_info(symbol).code_name[0]
            results.append(res)
    except:
        traceback.print_exc()
        print("fail on: {}".format(symbol))

print("选股结果：")
alldf = pd.DataFrame(results)
# df.to_excel("选股结果.xlsx", index=False)


In [11]:

# 如果是第一次使用需要设置 token
# set_token("手机号", "密码")

# 使用聚宽数据在创业板综指上选股
# symbols = get_index_stocks("399006.XSHE")
symbols = get_index_stocks("hs300")
freq = "d"

results = []
for symbol in tqdm(symbols, desc="selector"):
    res = {"is_match": False, "fx": None, "bi": None, "dt": None, "price": None, "fx_h": None, "fx_l": None,
           "name": "selector_bi02", "symbol": None, "三笔回调构成三买": False, "笔中枢三买选股": False, "name": None}
    try:
        kline = get_kline(symbol, end_date=datetime.now(), freq=freq, count=200)
        for selector in [selector_bi01, selector_bi02]:
            try:
                res = selector(kline, res)
            except:
                continue
        if res['is_match']:
            res['name'] = get_stock_info(symbol).code_name[0]
            results.append(res)
    except:
        traceback.print_exc()
        print("fail on: {}".format(symbol))

print("选股结果：")
hs300df = pd.DataFrame(results)
# df.to_excel("选股结果.xlsx", index=False)

<ipython-input-11-ea639650d9cd>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for symbol in tqdm(symbols, desc="selector"):


selector:   0%|          | 0/300 [00:00<?, ?it/s]

选股结果：


In [12]:
pd.set_option('display.max_columns', None)
hs300df

,is_match,fx,bi,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
0,True,"{'dt': 2021-01-06 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2021-01-06 00:00:00, 'fx_mark': 'd', 's...",2021-01-06,18.26,18.07,17.36,中国神华,sh.601088,True,False
1,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'g', 'f...","{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 's...",2020-12-31,22.69,22.45,21.26,长安汽车,sz.000625,True,False
2,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 's...",2020-12-29,42.75,38.78,36.63,科大讯飞,sz.002230,True,False
3,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'g', 'f...","{'dt': 2020-12-25 00:00:00, 'fx_mark': 'd', 's...",2020-12-29,23.23,26.25,23.39,上汽集团,sh.600104,True,False
4,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 's...",2020-12-30,26.06,21.78,20.90,中信特钢,sz.000708,True,False
5,True,"{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 's...",2020-12-31,22.81,20.83,19.54,江西铜业,sh.600362,True,False
6,True,"{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 's...",2020-12-29,4.71,4.24,4.12,中航资本,sh.600705,True,False


In [7]:
pd.set_option('display.max_rows', 1000)
zz500df

,is_match,fx,bi,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股
0,True,"{'dt': 2021-01-06 00:00:00, 'fx_mark': 'g', 'f...","{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 's...",2021-01-06,2.92,3.37,2.93,鹏都农牧,sz.002505,False,True
1,True,"{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-24 00:00:00, 'fx_mark': 'd', 's...",2020-12-30,7.27,6.65,6.30,晨鸣纸业,sz.000488,True,False
2,True,"{'dt': 2020-12-28 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-28 00:00:00, 'fx_mark': 'd', 's...",2020-12-28,62.04,54.18,52.30,洽洽食品,sz.002557,True,False
3,True,"{'dt': 2021-01-04 00:00:00, 'fx_mark': 'g', 'f...","{'dt': 2020-12-29 00:00:00, 'fx_mark': 'd', 's...",2021-01-04,7.94,8.88,8.20,燕京啤酒,sz.000729,True,False
4,True,"{'dt': 2021-01-05 00:00:00, 'fx_mark': 'g', 'f...","{'dt': 2020-12-30 00:00:00, 'fx_mark': 'd', 's...",2021-01-05,15.95,18.60,15.98,小康股份,sh.601127,True,False
5,True,"{'dt': 2021-01-06 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2020-12-28 00:00:00, 'fx_mark': 'd', 's...",2021-01-06,74.10,76.93,71.90,晶晨股份,sh.688099,True,False
6,True,"{'dt': 2021-01-06 00:00:00, 'fx_mark': 'g', 'f...","{'dt': 2020-12-31 00:00:00, 'fx_mark': 'd', 's...",2021-01-06,9.36,10.00,9.40,中粮科技,sz.000930,False,True
7,True,"{'dt': 2021-01-06 00:00:00, 'fx_mark': 'd', 'f...","{'dt': 2021-01-06 00:00:00, 'fx_mark': 'd', 's...",2021-01-06,4.81,4.81,4.57,华菱钢铁,sz.000932,True,False


In [13]:
hs300df.sort_values('dt', inplace=True)

In [14]:
hs300df.loc[:, ("dt", "price", "symbol", "name")]

,dt,price,symbol,name
2,2020-12-29,42.75,sz.002230,科大讯飞
3,2020-12-29,23.23,sh.600104,上汽集团
6,2020-12-29,4.71,sh.600705,中航资本
4,2020-12-30,26.06,sz.000708,中信特钢
1,2020-12-31,22.69,sz.000625,长安汽车
5,2020-12-31,22.81,sh.600362,江西铜业
0,2021-01-06,18.26,sh.601088,中国神华


In [15]:
hs300df[(hs300df['三笔回调构成三买'] == False) & (hs300df['笔中枢三买选股'] == True)]

,is_match,fx,bi,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股


In [16]:
hs300df[(hs300df['三笔回调构成三买'] == True) & (hs300df['笔中枢三买选股'] == True)]

,is_match,fx,bi,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股


In [8]:
zz500df.sort_values('dt', inplace=True)

In [9]:
zz500df.loc[:, ("dt", "price", "symbol", "name")]

,dt,price,symbol,name
2,2020-12-28,62.04,sz.002557,洽洽食品
1,2020-12-30,7.27,sz.000488,晨鸣纸业
3,2021-01-04,7.94,sz.000729,燕京啤酒
4,2021-01-05,15.95,sh.601127,小康股份
0,2021-01-06,2.92,sz.002505,鹏都农牧
5,2021-01-06,74.10,sh.688099,晶晨股份
6,2021-01-06,9.36,sz.000930,中粮科技
7,2021-01-06,4.81,sz.000932,华菱钢铁


In [10]:
zz500df[(zz500df['三笔回调构成三买'] == True) & (zz500df['笔中枢三买选股'] == True)]

,is_match,fx,bi,dt,price,fx_h,fx_l,name,symbol,三笔回调构成三买,笔中枢三买选股


In [ ]:
alldf.sort_values('symbol', inplace=True)

In [ ]:
alldf[(alldf['三笔回调构成三买'] == True) & (alldf['笔中枢三买选股'] == True)]